<a href="https://colab.research.google.com/github/priyadharsh73/airbnb_eda/blob/main/airbnb_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In-depth Exploratory Data Analysis of Airbnb Dataset

## Introduction
This notebook provides an in-depth exploratory data analysis (EDA) of the Airbnb dataset. The objective is to uncover insights and patterns in the data to better understand the factors influencing Airbnb listings.

### Airbnb Dataset Column Descriptions

| Column Name                      | Description                                                                 | Data Type   |
|----------------------------------|-----------------------------------------------------------------------------|-------------|
| id                               | Unique identifier for the listing                                           | int64       |
| name                             | Name of the listing                                                         | object      |
| host_id                          | Unique identifier for the host                                              | int64       |
| host_identity_verified           | Whether the host's identity is verified                                     | object      |
| host_name                        | Name of the host                                                            | object      |
| neighbourhood_group              | Grouping of neighbourhoods                                                  | object      |
| neighbourhood                    | Name of the neighbourhood                                                   | object      |
| lat                              | Latitude of the listing                                                     | float64     |
| long                             | Longitude of the listing                                                    | float64     |
| country                          | Country where the listing is located                                        | object      |
| country_code                     | Country code                                                                | object      |
| instant_bookable                 | Whether the listing is instantly bookable                                   | object      |
| cancellation_policy              | Cancellation policy of the listing                                          | object      |
| room_type                        | Type of room offered                                                        | object      |
| construction_year                | Year the property was constructed                                           | float64     |
| price                            | Price per night                                                             | float64     |
| service_fee                      | Service fee charged                                                         | float64     |
| minimum_nights                   | Minimum number of nights required to book                                   | float64     |
| number_of_reviews                | Total number of reviews received                                            | float64     |
| last_review                      | Date of the last review                                                     | datetime64  |
| reviews_per_month                | Average number of reviews per month                                         | float64     |
| review_rate_number               | Average rating of the listing                                               | float64     |
| calculated_host_listings_count   | Total number of listings by the host                                        | float64     |
| availability_365                 | Number of days the listing is available in a year                           | float64     |
| house_rules                      | House rules for the listing                                                 | object      |
| license                          | License number of the listing (if applicable)                               | object      |

## Data Loading and Preparation

### Import Libraries
First, import the necessary libraries for data manipulation, visualization, and mapping.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import MarkerCluster
import zipfile
import requests
from io import BytesIO

### Download and Extract ZIP File from GitHub

This step involves downloading the ZIP file containing the Airbnb dataset from GitHub and extracting the CSV file from it. The `requests` library is used to download the file, and the `zipfile` and `io` libraries are used to handle the extraction. The extracted CSV file is then read into a pandas DataFrame for further analysis.


In [20]:
# URL of the ZIP file on GitHub (direct download link)
url = 'https://github.com/priyadharsh73/airbnb_eda/raw/main/airbnb_dataset.zip'
filename = 'Airbnb_Open_Data.csv'

# Download the ZIP file
response = requests.get(url)
zip_file = BytesIO(response.content)

# Check if the response is valid
if response.status_code == 200:
    try:
        # Extract the ZIP file
        with zipfile.ZipFile(zip_file, 'r') as z:
            # List all files in the ZIP
            print(z.namelist())

            # Extract and read the CSV file
            with z.open(filename) as f:
                df = pd.read_csv(f, low_memory=False)

        # Display the first few rows of the DataFrame
    except zipfile.BadZipFile:
        print("Error: The file is not a valid ZIP file.")
else:
    print(f"Error: Failed to download the file. Status code: {response.status_code}")


['Airbnb_Open_Data.csv']


###Clean Column Names
- Use the `str.replace()` method to replace spaces with underscores.
- Use the `str.lower()` method to convert column names to lowercase.

Here’s the code to clean the column names:

In [21]:
# Clean column names by replacing spaces with underscores and converting to lowercase
df.columns = df.columns.str.replace(' ', '_').str.lower()

# Check the cleaned column names
print("Cleaned column names:")
print(df.columns)

Cleaned column names:
Index(['id', 'name', 'host_id', 'host_identity_verified', 'host_name',
       'neighbourhood_group', 'neighbourhood', 'lat', 'long', 'country',
       'country_code', 'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'price', 'service_fee', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'review_rate_number', 'calculated_host_listings_count',
       'availability_365', 'house_rules', 'license'],
      dtype='object')


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   name                            102349 non-null  object 
 2   host_id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host_name                       102193 non-null  object 
 5   neighbourhood_group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  object 
 10  country_code                    102468 non-null  object 
 11  instant_bookable                102494 non-null  object 
 12  cancellation_pol

### Misinterpreted Columns

Based on the `info()` function output, the following columns have been misinterpreted:

- **price**: Interpreted as `object`, should be `float`.
- **service_fee**: Interpreted as `object`, should be `float`.
- **minimum_nights**: Interpreted as `float64`, should be `int`.
- **number_of_reviews**: Interpreted as `float64`, should be `int`.
- **last_review**: Interpreted as `object`, should be `date`.
- **construction_year**: Interpreted as `float64`, should be `int`.

### Code to Correct Data Types

In [23]:
# Convert price and service_fee columns to numeric after removing the dollar sign and commas
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
df['service_fee'] = df['service_fee'].replace('[\$,]', '', regex=True).astype(float)

# Convert minimum_nights and number_of_reviews to integers, ignoring rows with missing values
df['minimum_nights'] = pd.to_numeric(df['minimum_nights'], errors='coerce').fillna(0).astype(int)
df['number_of_reviews'] = pd.to_numeric(df['number_of_reviews'], errors='coerce').fillna(0).astype(int)

# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

# Convert construction_year to integer
df['construction_year'] = df['construction_year'].astype('Int64')


The following columns had their data types corrected:
- `price` and `service_fee` were converted to numeric data types after removing the dollar signs and commas.
- `minimum_nights` and `number_of_reviews` were converted to integers, with missing values filled with 0.
- `last_review` was converted to datetime.
- `construction_year` was converted to integer.

In [24]:
# Summary of missing values
missing_values_summary = df.isna().sum()

# Display the summary of missing values
missing_values_summary

,0
id,0
name,250
host_id,0
host_identity_verified,289
host_name,406
neighbourhood_group,29
neighbourhood,16
lat,8
long,8
country,532


Insights on Missing Values
The dataset contains several columns with missing values. Here is a grouped summary of the missing values:

- **Identification and Location**:
  - `id`, `host_id`, `room_type`, `minimum_nights`, and `number_of_reviews` have no missing values.
  - `name`, `host_identity_verified`, `host_name`, `neighbourhood_group`, `neighbourhood`, `lat`, `long`, `country`, and `country_code` have a small to moderate number of missing values.

- **Booking and Policy**:
  - `instant_bookable`, `cancellation_policy`, `price`, and `service_fee` have a moderate number of missing values.
  - `construction_year` has some missing values.

- **Reviews and Availability**:
  - `last_review` and `reviews_per_month` have a significant number of missing values.
  - `review_rate_number`, `calculated_host_listings_count`, and `availability_365` have a moderate number of missing values.

- **Additional Information**:
  - `house_rules` has a large number of missing values.
  - `license` is almost entirely missing.

Handling these missing values appropriately is crucial for ensuring the quality and reliability of the analysis.

#### Dropping Insignificant Columns
- Dropped the columns `license` and `house_rules` due to a high percentage of missing values and their insignificance for the analysis.

In [25]:
# Drop insignificant columns
df.drop(columns=['license', 'house_rules'], inplace=True)

> After dropping these columns, the dataset is now more streamlined for further analysis.

#### Finding and Deleting Duplicates
Duplicates in the dataset were identified and removed based on the following important columns: `id`, `name`, `host_id`, `lat`, `long`, and `price`. This ensures that each listing is unique and prevents any potential bias or redundancy in the analysis.


In [26]:
# Count duplicates based on important columns
duplicate_count = df.duplicated(subset=['id', 'name', 'host_id', 'lat', 'long', 'price']).sum()

# Display the count of duplicates
print(f"Number of duplicate rows: {duplicate_count}")

# Drop duplicates based on important columns
df.drop_duplicates(subset=['id', 'name', 'host_id', 'lat', 'long', 'price'], keep='first', inplace=True)

# Summary of the dataset after removing duplicates
df.info()

Number of duplicate rows: 541
<class 'pandas.core.frame.DataFrame'>
Index: 102058 entries, 0 to 102057
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   id                              102058 non-null  int64         
 1   name                            101808 non-null  object        
 2   host_id                         102058 non-null  int64         
 3   host_identity_verified          101769 non-null  object        
 4   host_name                       101654 non-null  object        
 5   neighbourhood_group             102029 non-null  object        
 6   neighbourhood                   102042 non-null  object        
 7   lat                             102050 non-null  float64       
 8   long                            102050 non-null  float64       
 9   country                         101526 non-null  object        
 10  country_code                   

> The dataset contains a certain number of duplicate rows based on the important columns `id`, `name`, `host_id`, `lat`, `long`, and `price`.

After identifying the duplicates, they were removed to ensure that each listing is unique. This step helps in preventing any potential bias or redundancy in the analysis.


### Handling Missing Values
Based on the nature of each column, the following strategies were applied to handle missing values:

- **String Columns**: Filled with a placeholder "Unknown".
- **Categorical Columns**: Filled with the mode (most frequent value).
- **Numerical Columns**: Converted to numeric with `pd.to_numeric` and filled with the median value.
- **Date Columns**: Forward fill to maintain temporal sequence.
- **Critical Columns (lat, long)**: Rows with missing values were dropped.


In [27]:
# Fill missing values for string columns with a placeholder
df['name'].fillna('Unknown', inplace=True)
df['host_name'].fillna('Unknown', inplace=True)

# Fill missing values for categorical columns with the mode
df['host_identity_verified'].fillna(df['host_identity_verified'].mode()[0], inplace=True)
df['neighbourhood_group'].fillna(df['neighbourhood_group'].mode()[0], inplace=True)
df['neighbourhood'].fillna(df['neighbourhood'].mode()[0], inplace=True)
df['country'].fillna(df['country'].mode()[0], inplace=True)
df['country_code'].fillna(df['country_code'].mode()[0], inplace=True)
df['instant_bookable'].fillna(df['instant_bookable'].mode()[0], inplace=True)
df['cancellation_policy'].fillna(df['cancellation_policy'].mode()[0], inplace=True)

# Drop rows with missing values for lat and long
df.dropna(subset=['lat', 'long'], inplace=True)

# Fill missing values for numerical columns with the median
df['construction_year'] = pd.to_numeric(df['construction_year'], errors='coerce').fillna(df['construction_year'].median()).astype('Int64')
df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(df['price'].median())
df['service_fee'] = pd.to_numeric(df['service_fee'], errors='coerce').fillna(df['service_fee'].median())
df['reviews_per_month'] = pd.to_numeric(df['reviews_per_month'], errors='coerce').fillna(df['reviews_per_month'].median())
df['review_rate_number'] = pd.to_numeric(df['review_rate_number'], errors='coerce').fillna(df['review_rate_number'].median())
df['calculated_host_listings_count'] = pd.to_numeric(df['calculated_host_listings_count'], errors='coerce').fillna(df['calculated_host_listings_count'].median())
df['availability_365'] = pd.to_numeric(df['availability_365'], errors='coerce').fillna(df['availability_365'].median())

# Forward fill for last_review
df['last_review'].fillna(method='ffill', inplace=True)

# Summary of missing values after handling
missing_values_summary_after = df.isna().sum()

# Display the summary of missing values after handling
missing_values_summary_after

<ipython-input-27-8429efc3edb5>:27: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['last_review'].fillna(method='ffill', inplace=True)


,0
id,0
name,0
host_id,0
host_identity_verified,0
host_name,0
neighbourhood_group,0
neighbourhood,0
lat,0
long,0
country,0


> After applying these strategies, the dataset is now cleaner and more reliable for further analysis.